# Advanced Diagnostics for EPL Multiple Linear Regression

This notebook assumes that you have already:

1. Trained the model:
   ```bash
   python -m src.models.train --config configs/config.yaml
   ```
2. Run the diagnostics script (if implemented):
   ```bash
   python -m src.models.diagnostics --config configs/config.yaml
   ```

The diagnostics script is expected to create the following files under `reports/`:
- `vif.csv`
- `influence.csv`
- `anova.csv`
- `ols_summary.txt`
- `diagnostic_metrics.json`

This notebook demonstrates how to **inspect and visualize** those outputs.

In [ ]:
import pandas as pd
import json
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

reports_dir = Path('reports')
reports_dir.exists(), list(reports_dir.iterdir()) if reports_dir.exists() else 'reports dir not found'

## 1. Variance Inflation Factor (VIF)

VIF helps detect **multicollinearity** among predictors. Higher values mean
a predictor is more linearly related to other predictors.

- VIF ≈ 1: no correlation
- VIF 1–5: acceptable
- VIF > 5: potentially problematic
- VIF > 10: serious multicollinearity


In [ ]:
vif_path = reports_dir / 'vif.csv'
vif_df = pd.read_csv(vif_path)
vif_df.sort_values('vif', ascending=False).reset_index(drop=True)

## 2. Influence: Cook's Distance, Leverage, Standardized Residuals

We can visualize which observations are potentially influential or outliers.

In [ ]:
infl_path = reports_dir / 'influence.csv'
infl_df = pd.read_csv(infl_path)
infl_df.head()

In [ ]:
plt.figure(figsize=(6, 4))
sns.scatterplot(data=infl_df, x='leverage', y='std_resid')
plt.axhline(2, linestyle='--')
plt.axhline(-2, linestyle='--')
plt.title('Leverage vs Standardized Residuals')
plt.xlabel('Leverage')
plt.ylabel('Standardized Residual')
plt.tight_layout()
plt.show()

## 3. Diagnostic Metrics (AIC, BIC, PRESS, Durbin-Watson, Normality, BP Test)

These metrics summarize model quality, predictive performance, and whether
regression assumptions are violated.

In [ ]:
metrics_path = reports_dir / 'diagnostic_metrics.json'
with metrics_path.open('r') as f:
    metrics = json.load(f)

metrics

From here, you can:

- Compare AIC/BIC across different model specifications.
- Check if `durbin_watson` is close to 2 (no autocorrelation).
- Use normality p-value (`normal_ad_pvalue`) to assess residual normality.
- Use Breusch–Pagan p-values (`bp_lm_pvalue`, `bp_f_pvalue`) to detect heteroscedasticity.

You can also extend this notebook with:

- QQ-plots for residuals
- Histograms/density plots of residuals
- Highlighting points with large Cook's distance or leverage
